In [1]:
import time
import urllib.request
from optparse import OptionParser
import requests, zipfile, io
import gzip

In [2]:
start = time.time()
start

1589059093.0232387

In [3]:
#print('Beginning file GO download with urllib2...')
#urlGo = 'http://geneontology.org/gene-associations/'
#filename = "goa_human.gaf.gz"
#filenameInFolder = "goa_human_downloaded.gaf"
#outFilePath = filename[:-3]

In [4]:
#print('Beginning file HPO download with urllib2...')
##url = 'http://compbio.charite.de/jenkins/job/hpo.annotations.monthly/lastSuccessfulBuild/artifact/annotation/ALL_SOURCES_ALL_FREQUENCIES_genes_to_phenotype.txt'
#url = 'http://compbio.charite.de/jenkins/job/hpo.annotations/lastBuild/artifact/util/annotation/genes_to_phenotype.txt';
#urllib.request.urlretrieve(url, 'C:/Users/Aco/Desktop/annovar - jupyter/allHPODownload.txt')

In [5]:
output = open("output.txt", "r+")
outputNrna = open("outputNrnaExonic.txt", "r+")

In [6]:
#parse goa_human.gaf file with GO info
def getGOdata(filename):
    geneGO = {}
    count = 0
    with open(filename, 'r') as goaHuman:
        for line in goaHuman:
            pos = line.find("GO:")
            firstTab = line.find("\t")
            secondTab = line.find("\t", firstTab+1)
            thirdTab = line.find("\t", secondTab+1)
            gene = line[secondTab+1:thirdTab]
            val = thirdTab+1
            function = " "
            if pos != -1:
                go = line[pos:pos+10]
                
                #functionWhole = line[thirdTab:endTab] + '<br />'

                if gene in geneGO:
                    current = [currentGeneInfo[0] for currentGeneInfo in geneGO[gene]]
                    if go not in current:
                        geneGO[gene].append((go, function))
                    #geneGO[gene].append((go, functionWhole))
                else:
                    geneGO[gene] = [(go, function)]
                    #print(str(pos) + " :prvi" + go)

            #if count > 30:
               #break

    print (len(geneGO.keys()))
    return geneGO

In [7]:
#parse file with HPO info
def getHPOdata(filename):
    #phenotypes
    geneHpo = {}

    with open(filename, "r") as allHpo:
        for line in allHpo:
            pos = line.find("\t")
            if pos == -1:
                continue
            endOfGene = line.find("\t", pos+1)
            gene = line[pos+1:endOfGene]

            hpoStart = endOfGene+1
            hpoEnd = hpoStart+10
            hpo = line[hpoStart : hpoEnd]

            #phenotypeStart = hpoEnd+1
            #phenotypeEnd = line.find("\t", phenotypeStart+1)
            #phenotype = line[phenotypeStart : phenotypeEnd]
            phenotype = "fenotip"
            
            if gene in geneHpo:
                currentHpo = [currentGeneInfoH[0] for currentGeneInfoH in geneHpo[gene]]
                if hpo not in currentHpo:
                    geneHpo[gene].append(hpo)
            else:
                geneHpo[gene] = [hpo]
            #print(gene+" " + hpo  + " " + phenotype)
    return geneHpo


In [8]:
#parse info from annotation file
def getVariationsData(fileName):
    variations = {}
    allExonicGenes = ''
    geneInfo = {}

    with open(fileName, 'r') as varFile:
        for line in varFile:
            pos = line.find("exonic")
            if pos != -1:
                currentGenePositionEnd = line.find("\t", pos+7) #pos+7 is start
                currentGene = line[pos+7:currentGenePositionEnd]
                begin = -1
                #for i in range(0, 9): 
                #    val = line.find("\t", val + 1)
                #line[:val].replace("\t", "</td><td>")
                begin = line.find("\t")
                chro = line[:begin]
                end = line.find("\t", begin+1)
                startVar = line[begin+1:end]
                begin = end+1
                end = line.find("\t", begin)
                endVar = line[begin:end]
                begin = end+1
                end = line.find("\t", begin)
                ref = line[begin:end]
                begin = end+1
                end = line.find("\t", begin)
                alt = line[begin:end]
                #exonic, currentGene
                geneDetailEnd = line.find("\t", currentGenePositionEnd + 1)
                geneDetail = line[currentGenePositionEnd+1 : geneDetailEnd]
                
                exonicFuncEnd = line.find("\t", geneDetailEnd + 1)
                exonicFunc = line[geneDetailEnd + 1 : exonicFuncEnd]
                
                if currentGene in variations:
                    variations[currentGene].append((startVar, endVar, ref, alt, chro, geneDetail, exonicFunc))
                else:
                    variations[currentGene] = [(startVar, endVar, ref, alt, chro, geneDetail, exonicFunc)]
                
    return variations

In [9]:
def createOutput(fileName):
    
    goFile = "goa_human.gaf"
    hpoFile = "allHPODownload.txt"
    
    geneGO = getGOdata(goFile)
    geneHpo = getHPOdata(hpoFile)
    variations = getVariationsData(fileName)
    print (len(variations.keys()))
    
    outputFileName = "outputHtml" + fileName[:fileName.find('.')] + ".html"
    outputHtml = open(outputFileName, "w+")
    
    outputHtml.write("<!DOCTYPE html>\n")
    outputHtml.write("<html><body><head>")
    outputHtml.write("<script src='https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js'></script>")
    outputHtml.write("<script type=\"text/javascript\" charset=\"utf8\" src=\"https://cdn.datatables.net/1.10.20/js/jquery.dataTables.js\"></script>")
    outputHtml.write("<script src='scriptFile.js'></script>")
    outputHtml.write("<link rel=\"stylesheet\" type=\"text/css\" href=\"https://cdn.datatables.net/1.10.20/css/jquery.dataTables.css\"></head>")
    outputHtml.write("")
    outputHtml.write("<div id=\"showXYfunctions\">Show  <select id=\"showXFunctions\"><option value=\"5\">5</option><option value=\"10\">10</option><option value=\"15\">15</option><option value=\"20\">20</option><option value=\"25\">25</option><option value=\"30\">30</option></select>functions</div>")
    outputHtml.write("<div class=\"dataTables_length\"></div>")
    outputHtml.write("<table id=\"annotationTable\" class=\"display cell-border\" style=\"text-align:center\">")
    outputHtml.write("<thead><tr><th>Chr</th><th>Start</th><th>End</th><th>Ref</th><th>Alt</th><th>Func.refGene</th><th>Gene.refGene</th><th>GeneDetail.refGene</th><th>ExonicFunc.refGene</th><th>GO functions</th><th>HPO functions</th></tr></thead><tbody>")
    
    startURL = "<a href = \"https://www.ebi.ac.uk/QuickGO/term/"
    startURLhpo = "<a href = \"https://hpo.jax.org/app/browse/term/"
    aTagOpen = "<a"
    aTagClosed = "</a>"

    counterExonic = 0
    counterNrnaExonic = 0
    counterNo = 0
    counter = 0
    allExonicGenes = ""
    #list of all gene IDs for Uniprot convert
    with open(fileName, 'r') as file:
        for line in file:
            #line = file.readline()
            pos = line.find("exonic")
            if pos != -1:            
                currentGenePositionEnd = line.find("\t", pos+7) #pos+7 is start
                currentGene = line[pos+7:currentGenePositionEnd]
                if allExonicGenes.find(currentGene) == -1:
                    #goNo, hpoNo, ncRna = False

                    counter = counter + 1
                    allExonicGenes = allExonicGenes + currentGene
                    outputHtml.write("<tr><td>")
                    val = -1

                    #column chr
                    outputHtml.write(variations[currentGene][0][4] + "</td><td>")

                    #columns start, end, ref, alt
                    for i in range(0, 4):
                        outputHtml.write(" ".join(str(e[i] + "<br />") for e in variations[currentGene]))
                        outputHtml.write("</td><td>")

                    #column exonic
                    outputHtml.write("exonic</td><td>")

                    #column gene
                    outputHtml.write(currentGene + "</td><td>")

                    #geneDetail
                    outputHtml.write(variations[currentGene][0][5] + "</td><td>")

                    #exonic.Func                
                    outputHtml.write(" ".join(str(e[6] + "<br />") for e in variations[currentGene]))
                    outputHtml.write("</td><td>")

                    #if line[pos-1:pos] == "\t":
                        #counterExonic = counterExonic + 1
                        #output.write(currentGene)
                        #output.write(" ")
                    #else:
                        #counterNrnaExonic = counterNrnaExonic + 1
                        #outputNrna.write(currentGene)
                        #outputNrna.write(" ")
                        #ncRna = True                    

                    if currentGene in geneGO:
                        #outputHtml.write("*".join(str(startURL + e[0] +"\" target=\"_blank\">" + e[1] + aTagClosed)  for e in geneGO[currentGene])+ " ")#[:5]) + " ") #GO
                        if len(geneGO[currentGene]) > 5:
                            outputHtml.write(" ".join(str(startURL + e[0] +"\" target=\"_blank\">" + e[0] + "<br />" + aTagClosed)  for e in geneGO[currentGene][:5]) + " ") #GO
                            outputHtml.write(" ".join(str(startURL + e[0] +"\" target=\"_blank\" style=\"display:none\">" + e[0] + "<br /> " +aTagClosed)  for e in geneGO[currentGene][5:]) + " ") #GO
                            #outputHtml.write("<datalist id="+"GOs>" + " ".join(str("<option value=" + "<a href = https://www.ebi.ac.uk/QuickGO/term/" + e + ">" + e + "</a> >") for e in geneGO[currentGene][5:]) + "</datalist>") 
                        elif len(geneGO[currentGene]) >= 1:
                            outputHtml.write("<br />".join(str(startURL + e[0] +"\" target=\"_blank\">" + e[0] + aTagClosed)  for e in geneGO[currentGene][:5]) + " ") #GO

                    outputHtml.write("</td><td>")

                    if currentGene in geneHpo:
                        if len(geneHpo[currentGene]) > 5:
                            outputHtml.write(" ".join(str(startURLhpo + e +"\" target=\"_blank\">" + e + "<br />" + aTagClosed) for e in geneHpo[currentGene][:5]))
                            outputHtml.write(" ".join(str(startURLhpo + e +"\" target=\"_blank\" style=\"display:none\">" + e + "<br /> " +aTagClosed) for e in geneHpo[currentGene][5:]))
                        elif len(geneHpo[currentGene]) >=1:
                            outputHtml.write(" ".join(str(startURLhpo + e +"\" target=\"_blank\">" + e + aTagClosed) for e in geneHpo[currentGene][:5]))
                            #outputHtml.write("</td><td>")
                        #else:
                            #outputHtml.write("no info")
                            #hpoNo = True
                            #if goNo and ncRna:
                                #counterNo = counterNo + 1

                    outputHtml.write("</td></tr>\n")

    endOfTable = "<tfoot><tr><th>Chr</th><th>Start</th><th>End</th><th>Ref</th><th>Alt</th><th>Func.refGene</th><th>Gene.refGene</th><th>GeneDetail.refGene</th><th>ExonicFunc.refGene</th><th>GO functions</th><th>HPO functions</th></tr></tfoot></table>"
    outputHtml.write("</tbody>")
    outputHtml.write(endOfTable)
    outputHtml.write("</body></html>")


In [10]:
#createOutput("momAll.txt")
createOutput("dadAll.txt")
#createOutput("sonAll.txt")

19675
11317


In [12]:
end = time.time()
print("Duration: " + str(end-start) + "s")

Duration: 117.38722658157349s
